https://medium.com/swlh/web-scraping-python-requests-and-beautifulsoup-45d5f48f5a1

https://sparkbyexamples.com/spark/spark-read-text-file-rdd-dataframe/

https://medium.com/swlh/text2sql-in-spark-nlp-converting-natural-language-questions-to-sql-queries-on-scale-6ae9a9061d74

https://www.oreilly.com/library/view/learning-spark-2nd/9781492050032/ch04.html

In [ ]:
#!python -m pip install html2text

In [ ]:
#!python -m pip install mistletoe

https://github.com/webrecorder/warcio

In [ ]:
#!pip install warcio

In [ ]:
#!python -m pip install cdx_toolkit

In [ ]:
#%load_ext autoreload
#%autoreload 2

Run the cell below to import everything necessary for this analysis.

In [1]:
import json
import sys
import os.path
import re
import logging
import pandas as pd
import requests
import warcio
import mistletoe
import cdx_toolkit
import zlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from warcio import ArchiveIterator
from contextlib import closing
from html2text import HTML2Text
from bs4 import BeautifulSoup
from pathlib import Path
from IPython.display import HTML as HTML_raw, display
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
np.random.seed(0)

# Our corpus

In this analysis, we'll be working with alomost 100k different documents, each containing one single job AD.

These job AD plain text files are contained within the `\c\Users\renjm\Job-Posting-Big-Data\ana\dat\100k_jobADs` subdirectory, contained within the same folder as this analysis. Each job AD is stored in a single file, with files ranging from `1_2021JAN_jobAD.txt` to `99422_2021JAN_jobAD.txt`.

To make it easy to read in all of the documents, use a list comprehension to create a list containing the name of every single job AD plain text file below.

In [2]:
filesList = []
n_0 = 1
n_1 = 100000
for i in range(n_0, n_1):
    filepath = './dat/100k_jobADs/'+str(i)+'_2021JAN_jobAD.txt'
    if (os.path.exists(filepath)):
        filesList.append(str(i)+'_2021JAN_jobAD.txt')

#print(filesList)

next, create an empty DataFrame called `jobads_df`. As we read in job ads and store and clean them, we'll store them in this DataFrame.

In [3]:
jobads_df = pd.DataFrame()

next, let's import one job AD to see what our text looks like so that we can make sure we clean and tokenize it correctly.

read in and print out the `job description` from `11_2021JAN_jobAD.txt`. Use vanilla Python, no pandas needed.

93148 (113), 59721 (107)

In [4]:
filepick = './dat/100k_jobADs/59306_2021JAN_jobAD.txt' 
with open(filepick, encoding='utf-8', errors='ignore') as f:
    test_jobad = f.readlines()
    print(test_jobad)

['\n', ' Drivers \n', ' Warehouse \n', ' Hiring Event - Harriman, NY \n', ' Hiring Event - Lexington, SC \n', ' \n', ' LOCAL CDL-A TRUCK DRIVER JOBS WITH DAILY HOME TIME - Enfield, CT \n', ' SEARCH MORE JOBS __ \n', ' Hungry For A Better Truck Driving Career? Join The Martin Brower Private Fleet! \n', ' Martin Brower, a leading supply chain solutions provider for restaurants around the world, is now hiring CDL-A truck drivers for local opportunities. As part of one of the largest privately held companies in the United States, Martin Brower is perfectly equipped to offer CDL professionals incentives that drive their career forward: local routes, daily or nightly home time, 24/7 support and excellent earning potential! \n', " We're a People First company, driven to provide a culture that is focused on safety and committed to the well-being of our world-class team. If you're hungry for better truck driving opportunities, you'll find them here at Martin Brower! \n", ' *** THIS POSITION IS 

## Tokenizing our Data

Before we can create a bag of words or vectorize each document, we need to clean it up and split each job AD into an array of individual words. Computers are very particular about strings. If we tokenized our data in its current state, we would run into the following problems:

* Counting things that aren't actually words. 
* Punctuation and capitalization would mess up our word counts. To the Python interpreter, `love`, `Love`, `Love?`, and `Love\n` are all unique words, and would all be counted separately. We need to remove punctuation and capitalization, so that all words will be counted correctly.

Tokenization is pretty tedious if we handle it manually, and would probably make use of `regular expressions`. In order to keep this lab moving, we'll use a library function to clean and tokenize our data so that we can move onto vectorization.

Tokenization is a required task for just about any Natural Language Processing (NLP) task, so great industry-standard tools exist to tokenize things for us, so that we can spend our time on more important tasks without getting bogged down hunting every special symbol or punctuation in a massive dataset. For this analysis, we'll make use of the tokenizer in the amazing `nltk` library, which is short for *Natural Language Tool Kit*.

**NOTE**: `NLTK` requires extra installation methods to be run the first time certain methods are used. If `nltk` throws you an error about needing to install additional packages, follow the instructions in the error message to install the dependencies, and then re-run the cell.

Before we tokenize our job AD plain text files, we'll do only a small manual bit of cleaning. In the cell below, a function that allows us to make every word lowercase, remove newline characters `\n`, and all the following punctuation marks: `",.'?!"` is developed.

Furthermore, this function that not only takes in job AD plain text files having erratic symbols removed but also joins all of the lines into a single string. To sum up, the function, `tokenize_jobAD()`, is utilized to get a fully tokenized version of the job AD. We can test this function on `jobad` to ensure that the function works.

In [5]:
def tokenize_jobAD(jobad):
    subs = [('.',''),("'",""),('"',''),(',',''),('?',''),('!',''),('\n',''),('\t',''),('\r',''),('|',''),('/',' '),
            ('-',''),('(c)',''),('*',''),('(',''),(')',''),('&',''),(':',''),(';',''),('[',''),(']',''),('\\',''),
            ('â„¢',''),('%',''),('â£',''),('Â£0',''),('Â£',''),('>',''),('<',''),('=',''),('_',''),('__',''),('“',''),
            ('’',''),('–',''),('”',''),('—',''),('+','')]
    cleaned_jobad = []
    for line in jobad:
        for old, new in subs:
            line = line.replace(old, new).lower()
        cleaned_jobad.append(line)

    tokenized_jobad = ' '.join(cleaned_jobad).split()
    
    return tokenized_jobad

In [6]:
tokenized_test_jobad = tokenize_jobAD(test_jobad)
print(len(tokenized_test_jobad))
print(tokenized_test_jobad[:100000])

472
['drivers', 'warehouse', 'hiring', 'event', 'harriman', 'ny', 'hiring', 'event', 'lexington', 'sc', 'local', 'cdla', 'truck', 'driver', 'jobs', 'with', 'daily', 'home', 'time', 'enfield', 'ct', 'search', 'more', 'jobs', 'hungry', 'for', 'a', 'better', 'truck', 'driving', 'career', 'join', 'the', 'martin', 'brower', 'private', 'fleet', 'martin', 'brower', 'a', 'leading', 'supply', 'chain', 'solutions', 'provider', 'for', 'restaurants', 'around', 'the', 'world', 'is', 'now', 'hiring', 'cdla', 'truck', 'drivers', 'for', 'local', 'opportunities', 'as', 'part', 'of', 'one', 'of', 'the', 'largest', 'privately', 'held', 'companies', 'in', 'the', 'united', 'states', 'martin', 'brower', 'is', 'perfectly', 'equipped', 'to', 'offer', 'cdl', 'professionals', 'incentives', 'that', 'drive', 'their', 'career', 'forward', 'local', 'routes', 'daily', 'or', 'nightly', 'home', 'time', '24', '7', 'support', 'and', 'excellent', 'earning', 'potential', 'were', 'a', 'people', 'first', 'company', 'driven'

Now that we can tokenize our job AD plain text file, we can move onto vectorization.

## Count Vectorization

Machine Learning algorithms don't understand strings. However, they do understand math, which means they understand vectors and matrices. By Vectorizing the text, we just convert the entire text into a vector, where each element in the vector represents a different word. The vector is the length of the entire vocabulary -- usually, every word that occurs in the English language, or at least every word that appears in our corpus. Any given sentence can then be represented as a vector where all the vector is 1 (or some other value) for each time that word appears in the sentence.

`Count Vectorization` allows us to represent a sentence as a vector, with each element in the vector corresponding to how many times that word is used. Notice that when we vectorize a sentence this way, we lose the order that the words were in. This is the `Bag of Words` approach

In the cell below, create a function that takes in a tokenized, cleaned job AD plain text file and returns a count vectorized representation of it as a Python dictionary (aka: *Sparse Representation*). Add in an optional parameter called vocab that defaults to None. This way, if we are using a vocabulary that contains words not seen in the song, we can still use this function by passing it into the vocab parameter.

In [7]:
def count_vectorize(jobad, vocab=None):
    if vocab:
        unique_words = vocab
    else:
        unique_words = list(set(jobad))
    
    jobad_dict = {i:0 for i in unique_words}
    
    for word in jobad:
        jobad_dict[word] += 1
    
    return jobad_dict

In [8]:
test_vectorized = count_vectorize(tokenized_test_jobad)
print(test_vectorized)

{'started': 1, 'm': 1, 'time': 4, 'culture': 1, 'new': 4, 'working': 2, 'were': 1, 'delaware': 1, 'mileage': 1, 'provider': 1, 'nightly': 2, 'at': 6, 'virgin': 1, 'details': 1, 'position': 1, 'acceptable': 1, 'solutions': 1, '24': 3, 'prior': 1, 'dot': 2, 'arkansas': 1, 'team': 1, '$2910': 1, 'in': 4, 'schedule': 2, 'tennessee': 1, 'background': 1, 'consent': 1, 'commercial': 1, 'chain': 1, 'policy': 1, 'or': 5, 'the': 8, 'dakota': 2, 'to': 6, 'nevada': 1, 'with': 5, 'text': 1, 'usd': 1, 'wisconsin': 1, 'shifts': 1, 'enfield': 4, 'start': 1, 'illinois': 1, 'back': 1, 'no': 4, 'this': 1, 'hour': 1, 'maryland': 1, 'behalf': 1, 'a': 7, 'largest': 1, 'supply': 1, 'cdla': 5, 'united': 1, 'recruiter': 1, '52914': 1, 'south': 2, 'provided': 1, '1': 1, '2021': 1, 'delivery': 2, 'their': 1, 'lexington': 1, 'find': 1, 'connecticut': 2, 'o': 1, 'that': 2, 'distribution': 1, 'valid': 1, 'harriman': 1, 'one': 1, 'virginia': 2, 'z': 1, 'mvr': 1, 'autodialed': 1, 'prerecorded': 1, 'calls': 1, '15': 1

We've just successfully vectorized the first job AD plain text document! Now, let's look at a more advanced type of vectorization, TF-IDF!

In [9]:
statesList = ["United States of America","USA","AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS",
              "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA",
              "RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC","GU","MH","MP","PR","VI","Alabama","Alaska",
              "Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho",
              "Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts","Michigan",
              "Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","Hampshire","Jersey","Mexico","York",
              "Carolina","Dakota","Ohio","Oklahoma","Oregon","Pennsylvania","Rhode","Tennessee","Texas","Utah","Vermone",
              "Virginia","Washington","Wisconsin","Wyoming"]
itList = ["Engineer","Scientist","Developer","Analyst","Operator"]
qaList = ["Qualification","Qualifications","Certification","Certifications"]
euList = ["United Kingdom","UK","EU"]

check this text job AD text file to ensure it contains any key word(s) in `qaList`

In [10]:
def qa_usa_Count(filename, tokenized_jobad):
    found_qa = False
    counter_notusa = 0
    counter_qa = 0
    counter_usa = 0
    qaINjobList = []
    qaINusaDict = {}
    
    for eu in euList:
        if eu.lower() in tokenized_jobad.keys():
            counter_notusa += 1
            
    if counter_notusa == 0:
        for qa in qaList:
            if qa.lower() in tokenized_jobad.keys():
                counter_qa += 1
                found_qa = True
                qaINjobList.append(qa)
        if found_qa == True:
            for usa in statesList:
                if usa.lower() in tokenized_jobad.keys():
                    counter_usa += 1
                    qaINusaDict.setdefault(filename, []).append(usa)
            for qa in qaINjobList:
                qaINusaDict.setdefault(filename, []).append(qa)
                
    return counter_qa, counter_usa, qaINusaDict

In [11]:
qaNums_in_test_vectorized, usaNums_in_test_vectorized, qaDict_in_test_vectorized = qa_usa_Count('59306_2021JAN_jobAD', test_vectorized)
print(qaNums_in_test_vectorized)
print(usaNums_in_test_vectorized)
print(qaDict_in_test_vectorized)

1
51
{'59306_2021JAN_jobAD': ['CT', 'ID', 'IN', 'NY', 'OR', 'SC', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'Hampshire', 'Jersey', 'Mexico', 'York', 'Carolina', 'Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Washington', 'Wisconsin', 'Wyoming', 'Qualifications']}


In [12]:
def get_list_of_qaJobAds():
    n_0 = 1
    n_1 = 100000
    qaPassList = []
    okPassList = []
    okNotPassList = []
    for i in range(n_0, n_1):
        qaNums_in_test_vectorized = 0
        usaNums_in_test_vectorized = 0
        filepath = './dat/100k_jobADs/'+str(i)+'_2021JAN_jobAD.txt'
        if (os.path.exists(filepath)):
            filename = filepath.replace('./dat/100k_jobADs/','').replace('.txt','')
            with open(filepath, encoding='utf-8', errors='ignore') as f:
                test_jobad = f.readlines()
            tokenized_test_jobad = tokenize_jobAD(test_jobad)
            test_vectorized = count_vectorize(tokenized_test_jobad)
            qaNums_in_test_vectorized, usaNums_in_test_vectorized, qaDics_in_test_vectorized = qa_usa_Count(filename, test_vectorized)
            if (qaNums_in_test_vectorized > 0) and (usaNums_in_test_vectorized > 0):
                if (i%3 == 0):
                    qaPassList.append(qaDics_in_test_vectorized) 
                    okPassList.append(i)
            else:
                if (i%18 == 0): 
                    okNotPassList.append(i)
        else:
            continue
                
    return qaPassList, okPassList, okNotPassList

In [13]:
qaPassList, okPassList, okNotPassList = get_list_of_qaJobAds()
print(okPassList)

[12, 27, 96, 129, 210, 213, 285, 783, 786, 798, 810, 897, 909, 912, 993, 999, 1053, 1062, 1146, 1197, 1251, 1401, 1503, 1509, 1521, 1590, 1644, 1773, 1776, 1779, 1797, 1839, 2001, 2013, 2085, 2088, 2130, 2151, 2268, 2274, 2286, 2319, 2379, 2403, 2415, 2463, 2553, 2565, 2688, 2784, 2790, 2823, 2874, 2883, 2895, 2904, 2937, 3018, 3063, 3093, 3108, 3171, 3252, 3267, 3288, 3348, 3384, 3456, 3510, 3678, 3780, 3843, 3900, 3927, 3951, 3966, 3999, 4026, 4080, 4089, 4137, 4278, 4347, 4356, 4362, 4377, 4407, 4533, 4602, 4698, 4737, 4752, 4767, 4857, 4866, 4914, 4953, 4959, 5064, 5109, 5202, 5208, 5271, 5307, 5418, 5424, 5541, 5544, 5565, 5649, 5664, 5823, 5859, 5994, 6009, 6042, 6045, 6075, 6282, 6450, 6492, 6537, 6588, 6600, 6690, 6714, 6759, 6762, 6834, 6876, 6915, 6939, 6948, 6978, 7002, 7044, 7083, 7242, 7254, 7272, 7293, 7299, 7305, 7392, 7446, 7683, 7815, 7830, 7836, 7872, 7875, 7881, 7935, 7980, 8034, 8070, 8100, 8112, 8127, 8196, 8244, 8253, 8280, 8355, 8376, 8379, 8409, 8424, 8430, 8511

In [14]:
print(len(okPassList))
print(len(okNotPassList))

1798
1845


In [15]:
print(okNotPassList)

[54, 108, 234, 360, 378, 450, 504, 558, 612, 702, 756, 774, 864, 882, 954, 1008, 1098, 1116, 1152, 1224, 1242, 1350, 1368, 1422, 1458, 1530, 1584, 1656, 1674, 1692, 1782, 1800, 1836, 1854, 1872, 1908, 1998, 2016, 2034, 2124, 2178, 2250, 2340, 2412, 2448, 2502, 2700, 2736, 2772, 2880, 2970, 2988, 3024, 3096, 3276, 3294, 3312, 3330, 3402, 3420, 3474, 3564, 3582, 3654, 3726, 3744, 3798, 3834, 3942, 3960, 3978, 4014, 4068, 4122, 4176, 4230, 4284, 4320, 4410, 4554, 4662, 4680, 4734, 4770, 4860, 4950, 5058, 5148, 5166, 5184, 5220, 5238, 5274, 5382, 5454, 5472, 5490, 5526, 5562, 5706, 5760, 5922, 5958, 5976, 6048, 6084, 6120, 6174, 6192, 6246, 6264, 6300, 6336, 6354, 6372, 6390, 6426, 6462, 6642, 6678, 6696, 6768, 6822, 6894, 6966, 6984, 7020, 7164, 7182, 7218, 7380, 7398, 7506, 7524, 7578, 7632, 7704, 7740, 7758, 7794, 7848, 7866, 7938, 7956, 7974, 8028, 8046, 8082, 8154, 8172, 8190, 8334, 8352, 8406, 8442, 8496, 8550, 8568, 8586, 8604, 8640, 8712, 8802, 8820, 8856, 8892, 9072, 9090, 9216, 9

## TF-IDF Vectorization

TF-IDF stands for *Term Frequency*, *Inverse Document Frequency*. This is a more advanced form of vectorization that weighs each term in a document by how unique it is to the given document it is contained in, which allows us to summarize the contents of a document using a few key words. If the word is used often in many other documents, it is not unique, and therefore probably not too useful if we wanted to figure out how this document is unique in relation to other documents. Conversely, if a word is used many times in a document, but rarely in all the other documents we are considering, then it is likely a good indicator for telling us that this word is important to the document in question.

The formula TF-IDF uses to determine the weights of each term in a document is *Term Frequency* multiplied by *Inverse Document Frequency*, where the formula for Term Frequency is:

$$\large Term\ Frequency(t) = \frac{number\ of\ times\ t\ appears\ in\ a\ document} {total\ number\ of\ terms\ in\ the\ document} $$

Lets calculate term frequency for every term in a document.

In [16]:
def term_frequency(BoW_dict):
    total_word_count = sum(BoW_dict.values())
    
    for ind, val in BoW_dict.items():
        BoW_dict[ind] = val/total_word_count
    
    return BoW_dict

In [17]:
test = term_frequency(test_vectorized)
print(round(test['qualifications'],6))
print(list(test)[10:20])

0.002119
['nightly', 'at', 'virgin', 'details', 'position', 'acceptable', 'solutions', '24', 'prior', 'dot']


Now that we have this, we can easily calculate Inverse Document Frequency. In the cell below, complete the following function. this function should take in the list of dictionaries, with each item in the list being a bag of words representing the words in a different job AD plain text file. The function should return a dictionary containing the inverse document frequency values for each word.

The formula for Inverse Document Frequency is:

$$\large IDF(t) = log_e(\frac{Total\ Number\ of\ Documents}{Number\ of\ Documents\ with\ t\ in\ it})$$

In [18]:
def inverse_document_frequency(list_of_dicts):
    vocab_set = set()
    
    # Iterate through list of dfs and add index to vocab_set
    for d in list_of_dicts:
        for word in d.keys():
            vocab_set.add(word)
    
    # Once vocab set is complete, create an empty dictionary with a key for each word and value of 0.
    full_vocab_dict = {i:0 for i in vocab_set}
    
    # Loop through each word in full_vocab_dict
    for word, val in full_vocab_dict.items():
        docs = 0
        
        # Loop through list of dicts.  Each time a dictionary contains the word, increment docs by 1
        for d in list_of_dicts:
            if word in d.keys():
                docs += 1
        
        # Now that we know denominator for equation, compute and set IDF value for word
        full_vocab_dict[word] = np.log((len(list_of_dicts)/float(docs)))
    
    return full_vocab_dict

## Computing TF-IDF

Now that we can compute both `Term Frequency` and `Inverse Document Frequency`, computing an overall TF-IDF value is simple! All we need to do is multiply the two values.

In the cell below, complete the tf_idf() function. This function should take in a list of dictionaries, just as the inverse_document_frequency() function did. This function returns a new list of dictionaries, with each dictionary containing the `tf-idf vectorized representation` of a corresponding job AD plain text document.

NOTE: Each document should contain the full vocabulary of the entire combined corpus.

In [25]:
def tf_idf(list_of_dicts, dim=2000):
    
    # Create empty dictionary containing full vocabulary of entire corpus
    doc_tf_idf = {}
    idf = inverse_document_frequency(list_of_dicts)
    full_vocab_list = {i:0 for i in list(idf.keys())}
    
    # Create tf-idf list of dictionaries, containing a dictionary that will be updated for each document
    tf_idf_list_of_dicts = []
    
    # Now, compute tf and then use this to compute and set tf-idf values for each document
    for doc in list_of_dicts:
        doc_tf = term_frequency(doc)
        
        for qa in qaList:
            if qa.lower() in doc_tf.keys():
                doc_tf_idf[qa.lower()] = round(doc_tf[qa.lower()] * idf[qa.lower()],6)
                
        for word in doc_tf.keys():
            if word not in qaList:
                if (len(doc_tf_idf) < dim):
                    doc_tf_idf[word] = round(doc_tf[word] * idf[word],6)
                else: break
            else: continue
                    
        tf_idf_list_of_dicts.append(doc_tf_idf)
    
    return tf_idf_list_of_dicts

## Vectorizing All Documents

Now that we've created all the necessary helper functions, we can load in all of our documents and run each through the vectorization pipeline we've just created.

In the cell below, complete the `main()` function. This function should take in a list of file names (provided for you in the `filenames` list we created at the start), and then:

* Read in each document
* Tokenize each document
* Convert each document to a bag of words (dictionary representation)
* Return a list of dictionaries vectorized using tf-idf, where each dictionary is a vectorized representation of a document

In [26]:
def main(okList):
    
    # Iterate through list of filenames and read each in
    count_vectorized_all_documents = []
    for fileNo in okList:
        filepath = './dat/100k_jobADs/'+str(fileNo)+'_2021JAN_jobAD.txt'
        if (os.path.exists(filepath)):
            with open(filepath, encoding='utf-8', errors='ignore') as f:
                raw_data = f.readlines()
                
        # Clean and tokenize raw text
        tokenized = tokenize_jobAD(raw_data)
        
        # Get count vectorized representation and store in count_vectorized_all_documents  
        count_vectorized_document = count_vectorize(tokenized)
        count_vectorized_all_documents.append(count_vectorized_document)
    
    # Now that we have a list of BoW respresentations of each job AD, create a tf-idf representation of everything
    tf_idf_all_docs = tf_idf(count_vectorized_all_documents, 2000)
    
    return tf_idf_all_docs

In [27]:
tf_idf_okPass_docs = main(okPassList)
print(list(tf_idf_okPass_docs[0])[:10])

['qualifications', 'started', 'new', 'admin', 'working', 'advertise', 'us', 'login', '2017may', 'what']


In [28]:
print(tf_idf_okPass_docs[0])

{'qualifications': 0.000451, 'started': 0.00379, 'new': 0.006218, 'admin': 0.00238, 'working': 0.010475, 'advertise': 0.00562, 'us': 0.000526, 'login': 0.008513, '2017may': 0.011095, 'what': 0.000772, 'tips': 0.007324, 'investment': 0.000545, 'skills': 0.001877, 'opportunity': 0.001953, 'in': 1.6e-05, 'kpmg': 0.000701, 'five': 0.002823, 'good': 0.001112, 'policy': 0.001102, 'letter': 0.004379, 'the': 0.000756, 'employ': 0.001654, 'copyright': 0.007638, 'to': 0.000179, 'with': 0.000237, 'moving': 0.014491, 'recruiting': 0.004295, 'start': 0.00214, 'its': 0.001845, 'facts': 0.004946, 'existing': 0.004048, 'we': 0.001519, 'a': 0.000233, 'locations': 0.00699, 'my': 0.001797, 'businesses': 0.009941, 'shortage': 0.006549, 'find': 0.000343, 'post': 0.00085, 'privacy': 0.000262, 'getting': 0.003299, 'top': 0.001362, 'want': 0.013765, 'below': 0.002008, 'cover': 0.004265, 'links': 0.002596, 'people': 0.000884, 'jobs': 0.003126, 'leading': 0.001614, 'home': 0.001378, 'cv': 0.001036, 'by': 0.0005

In [29]:
tf_idf_okNotPass_docs = main(okNotPassList)
print(list(tf_idf_okNotPass_docs[0])[:10])

['were', 'details', 'userid', 'categories', '37', 'team', 'areas', 'share', 'quality', '3g']


In [30]:
print(tf_idf_okNotPass_docs[0])

{'were': 0.001509, 'details': 0.001944, 'userid': 0.001297, 'categories': 0.002458, '37': 0.000967, 'team': 0.000579, 'areas': 0.029701, 'share': 0.000462, 'quality': 0.001831, '3g': 0.000181, 'expensive': 0.001652, 'interface': 0.005521, 'determine': 0.000312, 'its': 0.001198, '100': 0.003316, 'harddrive': 0.001297, 'monetise': 0.001297, '01000ms': 0.001297, 'who': 0.000835, 'retrieved': 0.002054, '08': 0.006432, 'db2': 0.00619, 'mind': 0.000528, 'privacy': 5.2e-05, 'hires': 0.004855, 'project': 0.001374, 'specialized': 0.003222, 'channels': 0.016245, 'distinct': 0.002954, 'outofbox': 0.000831, 'allows': 0.002077, 'set': 0.000493, 'onload': 0.003892, 'build': 0.000207, 'then': 0.001646, 'rbac': 0.001297, 'excludes': 0.004039, 'red': 0.001894, 'exits': 0.005246, 'paid': 0.007736, 'methodology': 0.009644, 'social': 0.001453, 'objectives': 0.013624, 'represe': 0.001297, 'four': 0.000729, 'servers': 0.007812, 'columnar': 0.001297, 'example': 0.000699, 'snippet': 0.00078, '‘ghost': 0.00259

## Visualizing our Vectorizations

Now that we have a tf-idf representation of each document, we can move on to the fun part -- visualizing everything!

In the cell below, examine our dataset to figure out how many dimensions our dataset has.

**HINT**: Remember that every word is its own dimension!

In [31]:
num_okPass_dims = len(tf_idf_okPass_docs[0])
print("Number of okPass Dimensions: {}".format(num_okPass_dims))

Number of okPass Dimensions: 46681


In [32]:
num_okNotPass_dims = len(tf_idf_okNotPass_docs[0])
print("Number of okNotPass Dimensions: {}".format(num_okNotPass_dims))

Number of okNotPass Dimensions: 55130


There are too many dimensions for us to visualize! In order to make it understandable to human eyes, we'll need to reduce it to 2 or 3 dimensions.

To do this, we'll use a technique called **t-SNE** (short for `t-Stochastic Neighbors Embedding`). This is too complex for us to code ourselves, so we'll make use of scikit-learn's implementation of it.

First, we need to pull the words out of the dictionaries stored in `tf_idf_*_docs` so that only the values remain, and store them in lists instead of dictionaries. This is because the t-SNE only works with array-like objects, not dictionaries.

In the cell below, create a list of lists that contains a list representation of the values of each of the dictionaries stored in `tf_idf_*_docs`. The same structure should remain -- e.g. the first list should contain only the values that were in the first dictionary in `tf_idf_*_docs`, and so on.

In [33]:
tf_idf_vals_okPass_list = []

for i in tf_idf_okPass_docs:
    tf_idf_vals_okPass_list.append(list(i.values()))

In [34]:
print(tf_idf_vals_okPass_list[0][:10])

[0.000451, 0.00379, 0.006218, 0.00238, 0.010475, 0.00562, 0.000526, 0.008513, 0.011095, 0.000772]


In [35]:
tf_idf_vals_okNotPass_list = []

for i in tf_idf_okNotPass_docs:
    tf_idf_vals_okNotPass_list.append(list(i.values()))

In [36]:
print(tf_idf_vals_okNotPass_list[0][:10])

[0.001509, 0.001944, 0.001297, 0.002458, 0.000967, 0.000579, 0.029701, 0.000462, 0.001831, 0.000181]


Now that we have only the values, we can use the `TSNE()` class from `sklearn` to transform our data appropriately. In the cell below, instantiate `TSNE()` with `n_components=3`. Then, use the created object's `.fit_transform()` method to transform the data stored in `tf_idf_vals_list` into 3-dimensional data. Then, inspect the newly transformed data to confirm that it has the correct dimensionality.

In [37]:
t_sne_object_3d = TSNE(n_components=3)

In [38]:
transformed_okPass_data_3d = t_sne_object_3d.fit_transform(tf_idf_vals_okPass_list)
transformed_okPass_data_3d

array([[ 85.301476  ,  37.45989   , -16.690498  ],
       [  0.87214804,  95.36917   , -18.974379  ],
       [ 85.63241   , -10.486746  ,  38.47167   ],
       ...,
       [ 36.93424   , -54.225964  ,  37.172462  ],
       [ 36.93424   , -54.225964  ,  37.172462  ],
       [ 45.48997   , -37.06673   ,  43.73888   ]], dtype=float32)

In [39]:
transformed_okNotPass_data_3d = t_sne_object_3d.fit_transform(tf_idf_vals_okNotPass_list)
transformed_okNotPass_data_3d

array([[ -22.583124  ,   83.22133   ,   73.05571   ],
       [-114.63295   ,    0.91526306,  -49.244106  ],
       [  49.31479   ,   35.68485   ,   17.494698  ],
       ...,
       [  23.564945  ,  -45.645256  ,   10.812007  ],
       [ -17.968712  ,  -37.898155  ,  -24.913895  ],
       [ -32.50496   ,  -12.681629  ,   10.23064   ]], dtype=float32)

We'll also want to check out how the visualization looks in 2d. Repeat the process above, but this time, instantiate `TSNE()` with 2 components instead of 3. Again, use `.fit_transform()` to transform the data and store it in the variable below, and then inspect it to confirm the transformed data has only 2 dimensions.

In [40]:
t_sne_object_2d = TSNE(n_components=2)

In [41]:
transformed_okPass_data_2d = t_sne_object_2d.fit_transform(tf_idf_vals_okPass_list)
transformed_okPass_data_2d

array([[ 2.2394907 ,  0.25365013],
       [-0.7960829 ,  1.5926657 ],
       [ 0.156305  , -0.20800272],
       ...,
       [-0.99436086, -2.095965  ],
       [-0.9943608 , -2.0959654 ],
       [-1.0356336 , -2.0724096 ]], dtype=float32)

In [42]:
transformed_okNotPass_data_2d = t_sne_object_2d.fit_transform(tf_idf_vals_okNotPass_list)
transformed_okNotPass_data_2d

array([[-0.5541711 ,  0.8560095 ],
       [ 1.0152581 ,  1.8046672 ],
       [ 0.17692994,  1.6973977 ],
       ...,
       [ 1.1168358 ,  1.7371367 ],
       [-0.4996809 ,  0.05110619],
       [-1.7827904 ,  0.46007052]], dtype=float32)